In [1]:
import numpy as np
import pandas as pd

data=pd.read_csv('0000000000002747_training_twitter_x_y_train.csv')

test_data=pd.read_csv('0000000000002747_test_twitter_x_test.csv')

In [11]:
#cleaning the data (remove stop word and punctuation)
from nltk.corpus import stopwords
stop=stopwords.words('english')  

import string
punctuations=list(string.punctuation)
print(punctuations)
a=['']
stop=stop+punctuations+a

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


In [32]:
data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [33]:
twiter_categories=set(data['airline_sentiment'])
twiter_categories

{'negative', 'neutral', 'positive'}

In [36]:
documents=[]
test_documents=[]


for category in twiter_categories: 
    
    category_rows=(category==data['airline_sentiment'])
    
    x_train_current=data[category_rows]
    print(len(x_train_current),category)
    x_train_current.reset_index(drop=True,inplace=True)
    for i in range(len(x_train_current)):
        documents.append((x_train_current['text'][i],category))
import random
random.shuffle(documents)
print(len(documents))

for i in range(len(test_data)):
    test_documents.append((test_data['text'][i],'none'))
print(len(test_documents))

2327 neutral
6851 negative
1802 positive
10980
3660


In [37]:
new_documents=[(doc.split(' '),category) for doc,category in documents]
print(new_documents[0:2])

new_test_documents=[(doc.split(' '),category) for doc,category in test_documents]
#print(new_test_documents)

[(['@southwestair', 'Flight', '4146', 'Phi', 'to', 'Den', 'was', 'staffed', 'by', 'a', 'great', 'crew.', '#freecomedyshow', '#newlifetimecustomer'], 'positive'), (['@AmericanAir', 'i', 'ordered', 'it', 'as', 'i', 'always', 'do.', 'But', 'on', 'a', '9hour', 'flight', 'delayed', 'for', '4', 'hours,', 'it', 'was', 'worse', 'than', 'ever', 'before', 'when', 'you', 'forgot', 'my', 'meal.'], 'negative')]


In [39]:
#lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
#part of speech
from nltk import pos_tag

In [40]:
from nltk.corpus import wordnet
#convert post_tag to what we require for lemmatizer
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADJ
    else:
        return wordnet.NOUN

In [41]:
#words - available in particular document
#lemmetization
def clean_review(words):
    output_words=[]
    for w in words:
        if w.lower() not in stop:
            pos=pos_tag([w])
            
            clean_word=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [43]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL PC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [45]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\DELL
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [46]:
new_documents2=[(clean_review(document),category) for document,category in new_documents]
print(new_documents2[0:2])

new_test_documents2=[(clean_review(document),category) for document,category in new_test_documents]
print(new_test_documents2[0:2])

[(['@southwestair', 'flight', '4146', 'phi', 'den', 'staffed', 'great', 'crew.', '#freecomedyshow', '#newlifetimecustomer'], 'positive'), (['@americanair', 'order', 'always', 'do.', '9hour', 'flight', 'delayed', '4', 'hours,', 'bad', 'ever', 'forgot', 'meal.'], 'negative')]
[(['@americanair', 'car', 'gng', 'dfw.', 'pulled', '1hr', 'ago', 'icy', 'roads.', 'on-hold', 'aa', 'since', '1hr.', "can't", 'reach', 'arpt', 'aa2450.', 'wat', '2', 'do?'], 'none'), (['@americanair', 'all,', 'plane', 'didn’t', 'land', 'identical', 'worse)', 'condition', 'grk', 'accord', 'metars.'], 'none')]


In [51]:
#Building feature set
x_train_documents=new_documents2
x_test_documents=new_test_documents2
total_words=[]
for doc in x_train_documents:
    total_words+=doc[0]

In [54]:
#frequency distribution 
import nltk
freq=nltk.FreqDist(total_words)
common=freq.most_common(3000)# we have frequency corresponding to each word(feature)
features=[i[0] for i in common]

In [56]:
#words- available in particular document
def get_feature_dict(words):
    get_features={}
    words_set=set(words)
    for w in features:
        get_features[w]= w in words_set # store whether that feature/word(w) lie in that document or not
    return get_features

In [57]:
#Required data format to train the NaiveBayesClassifier
training_data=[(get_feature_dict(doc),category)  for doc,category in x_train_documents ]
testing_data=[get_feature_dict(doc)  for doc,category in x_test_documents ]

In [61]:
from nltk.classify import NaiveBayesClassifier

classifier=NaiveBayesClassifier.train(training_data)

In [62]:
y_pred=[]
for i in range(len(testing_data)):
    y_pred.append(classifier.classify(testing_data[i]))
y_pred

['negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',

In [63]:
import pandas as pd
df=pd.DataFrame(y_pred)
df.to_csv('TwiterAnalysis.csv',header=False,index=False)
classifier.show_most_informative_features(15)

Most Informative Features
                     :-) = True           positi : negati =     41.8 : 1.0
                    you! = True           positi : negati =     39.3 : 1.0
               wonderful = True           positi : negati =     34.2 : 1.0
                   kudos = True           positi : negati =     34.2 : 1.0
                      hr = True           negati : positi =     29.6 : 1.0
                favorite = True           positi : negati =     29.1 : 1.0
              @jetblue's = True           neutra : negati =     24.5 : 1.0
                   &lt;3 = True           positi : negati =     24.1 : 1.0
                   great = True           positi : neutra =     23.8 : 1.0
                awesome! = True           positi : negati =     23.6 : 1.0
                   thank = True           positi : negati =     23.4 : 1.0
                      :) = True           positi : negati =     22.6 : 1.0
                   much! = True           positi : negati =     22.0 : 1.0